Note: you may need to restart the kernel to use updated packages.


ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [8]:
import scipy
import scipy.io
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import savemat
import os
from feature_extraction import features_estimation

In [15]:
num_of_subject = 54

In [57]:
# Define the function to plot a graph
def plot_signal_modification(time, original_signal, time1, new_signal):
    """
    Function to plot a simple graph.

    Parameters:
    - x: List or array of x-values
    - y: List or array of y-values
    - title: Title of the graph (default: "Graph")
    - xlabel: Label for the x-axis (default: "X-axis")
    - ylabel: Label for the y-axis (default: "Y-axis")
    """

    # Plotting the original EMG signal
    plt.figure(figsize=(10, 5))
    plt.plot(time, original_signal, color='b', linewidth=2)
    plt.plot(time1, new_signal, color='r', linewidth=1)

    # Adding labels and title
    plt.xlabel("Time (s)")
    plt.ylabel("EMG Signal (mV)")
    plt.title("EMG Signal Over Time")
    plt.grid(True)


    # time_resampled = np.linspace(0, time[-1], target_length)
    
    plt.show()


In [81]:
def uniformize_data_subject(file_path, sub_num):


    num_sensors = 24
    target_length = 15376
    sampling_rate = 5120


    # Load the MATLAB file
    mat_data = scipy.io.loadmat(file_path)
    data = mat_data['subjectsSpecificSetGestures']
    set_of_all_gesture_by_subject = np.array(data)

    gestures = set_of_all_gesture_by_subject.shape[1]
    repetitions = set_of_all_gesture_by_subject.shape[0]


    # we access all repetitions of the 7 types of hand gestures
    for i in range(repetitions): # shape (28,)

        # we access a specific gesture type
        for j in range(gestures):

            # hand_gesture_type contains the electrovolts reading of the 24 sensors placed on the subjects arms at the ith repetition of the jth hand gesture type
            hand_gesture_type = set_of_all_gesture_by_subject[i,j]    # shape(....,24)
            
            # we check if the sampling length of the signals of this gesture matches the desired length
            if hand_gesture_type.shape[0] != target_length:

                # if it isn't we create a new array of the appropriate size
                new_gesture = np.zeros((target_length, num_sensors))

                # for all sensors we modify the expand the length of the gesture recorded
                for k in range(num_sensors):
                    sample_to_alter = hand_gesture_type[:,k]  # shape (..., 1)
                    resampled_signal = signal.resample(sample_to_alter , target_length)
                    new_gesture[:,k] = resampled_signal

                    # time = np.arange(len(sample_to_alter)) / sampling_rate
                    # time2 = np.arange(len(resampled_signal)) / sampling_rate
                    # plot_signal_modification(time, sample_to_alter, time2, resampled_signal)
                
                set_of_all_gesture_by_subject[i,j] = new_gesture

    # Example data
    output_data = {'processed_data': set_of_all_gesture_by_subject}



    # Specify directory and filename
    output_dir = "E:\\Chris\\putEMG Project\\processed_data"
    os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
    output_path = os.path.join(output_dir, 'processed_gestures_subject'+ str(sub_num)+ '.mat')

    # Save the file
    savemat(output_path, output_data)
    


In [32]:
subject = 3
file_path = "E:\Chris\putEMG Project\processed_data\processed_gestures_subject"+ str(subject)+ ".mat"
print(file_path)

mat_data = scipy.io.loadmat(file_path)
data = mat_data['processed_data']
set_of_all_gesture_by_subject = np.array(data)

gestures = set_of_all_gesture_by_subject.shape[1]
repetitions = set_of_all_gesture_by_subject.shape[0]



test = set_of_all_gesture_by_subject[:,0][1][:,0]


# EMG Feature Extraction
emg_features, features_names = features_estimation(test, "0",
                                                   5120, 20000, 100)


print(emg_features)



# uniformize_data_subject(file_path, subject)

E:\Chris\putEMG Project\processed_data\processed_gestures_subject3.mat
EMG features were from channel 0 extracted successfully
Empty DataFrame
Columns: []
Index: [VAR, RMS, IEMG, MAV, LOG, WL, ACC, DASDV, ZC, WAMP, MYOP, FR, MNP, TP, MNF, MDF, PKF, WENT]


In [6]:
# for subject in range(1,num_of_subject):
#     if subject not in [0,1,2,21,28,32,37,40,41,44,52]:
#         file_path = "E:\Chris\putEMG Project\subject_gestures\subject"+ str(subject)+ "_gestures.mat"
#         print(file_path)
#         uniformize_data_subject(file_path, subject)



    